In [6]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [7]:
# Read recipe inputs
days_till_blooming = dataiku.Dataset("days_till_blooming")
df = days_till_blooming.get_dataframe()

In [8]:
df.head()

,年月日,年月日_parsed,年,日照時間,場所,降水量合計,平均気温,最高気温,最低気温,状況,開花日,開花日_parsed,年初からの日数,開花年,開花日までの日数
0,1991/1/1,1991-01-01 00:00:00+00:00,1991,2.3,大分,0.0,12.5,16.4,10.0,未開花,1991-03-28 00:00:00+00:00,1991-03-28 00:00:00+00:00,86.0,1991.0,86.0
1,1991/1/2,1991-01-02 00:00:00+00:00,1991,3.7,大分,3.0,10.2,12.8,6.5,未開花,1991-03-28 00:00:00+00:00,1991-03-28 00:00:00+00:00,86.0,1991.0,85.0
2,1991/1/3,1991-01-03 00:00:00+00:00,1991,7.7,大分,0.0,5.7,8.6,3.2,未開花,1991-03-28 00:00:00+00:00,1991-03-28 00:00:00+00:00,86.0,1991.0,84.0
3,1991/1/4,1991-01-04 00:00:00+00:00,1991,4.4,大分,0.0,6.1,9.7,2.1,未開花,1991-03-28 00:00:00+00:00,1991-03-28 00:00:00+00:00,86.0,1991.0,83.0
4,1991/1/5,1991-01-05 00:00:00+00:00,1991,0.7,大分,0.0,4.6,6.4,2.6,未開花,1991-03-28 00:00:00+00:00,1991-03-28 00:00:00+00:00,86.0,1991.0,82.0


In [9]:
df.sort_values([u'場所', u'開花日'], ascending=True, inplace=True)

In [10]:
s_ave = df.groupby([u'場所'])[u'年初からの日数'].expanding().mean().values

In [11]:
df[u'年初からの日数_暦年平均']=s_ave

In [0]:
df = df.sort_values([u'場所', u'年月日_parsed'], ascending=False).reset_index(drop=True)

In [0]:
df.head()

In [0]:
length = 30

In [0]:
var_to_vectorize = [u'平均気温', u'最高気温', u'最低気温', u'降水量合計', u'日照時間']

In [0]:
new_df = pd.DataFrame()
for city in df[u'場所'].unique():
    print(city)
    df_sub = df.loc[df[u'場所']==city].reset_index(drop=True)
    for var in var_to_vectorize:
        print(var)
        colname = var + u"推移"
        df_sub[colname] = np.nan
        long_vec = df_sub[var].values.tolist()

        for i in range(0, df_sub.shape[0]):
            small_vec = long_vec[i+1:i+length+1][::-1]
            if len(small_vec) == length:
                df_sub.loc[i, colname]=str(small_vec)
            else: break
    new_df = pd.concat([new_df, df_sub], ignore_index=True)

In [0]:
new_df = new_df[~new_df[u'平均気温推移'].isna()]

In [0]:
new_df.isna().sum()

In [0]:
vectorized_past = dataiku.Dataset("vectorized_past")
vectorized_past.write_with_schema(new_df)